In [ ]:
#!/usr/bin/env python
# coding: utf-8
# example usage: python compute_nhd_routing_SingleSeg.py -v -t -w -n Mainstems_CONUS
# python compute_nhd_routing_SingleSeg_v02.py --test -t -v --debuglevel 1
# python compute_nhd_routing_SingleSeg_v02.py --test-full-pocono -t -v --debuglevel 1


# -*- coding: utf-8 -*-
"""NHD Network traversal

A demonstration version of this code is stored in this Colaboratory notebook:
    https://colab.research.google.com/drive/1ocgg1JiOGBUl3jfSUPCEVnW5WNaqLKCD

"""
## Parallel execution
import os
import sys
import time
import numpy as np
import argparse
import pathlib
import glob
import pandas as pd
from collections import defaultdict
from functools import partial
from joblib import delayed, Parallel
from itertools import chain, islice
from operator import itemgetter



ENV_IS_CL = False
if ENV_IS_CL:
    root = pathlib.Path("/", "content", "t-route")
elif not ENV_IS_CL:
    #root = pathlib.Path("../..").resolve()
    # sys.path.append(r"../python_framework_v02")
    # TODO: automate compile for the package scripts
    #sys.path.append("fast_reach")
    root = os.path.dirname(os.path.dirname(os.path.abspath('')))
    sys.path.append(os.path.join(root, r'src', r'python_framework_v02','troute'))
    sys.path.append(os.path.join(root, r'src', r'python_routing_v02','fast_reach'))


import troute.nhd_network_utilities_v02 as nnu
import mc_reach
import troute.nhd_network as nhd_network
import troute.nhd_io as nhd_io
import build_tests  # TODO: Determine whether and how to incorporate this into setup.py
# import troute.nhd_network_augment as nna

"""
def writetoFile(file, writeString):
    file.write(writeString)
    file.write("\n")


def constant_qlats(index_dataset, nsteps, qlat):
    q = np.full((len(index_dataset.index), nsteps), qlat, dtype="float32")
    ql = pd.DataFrame(q, index=index_dataset.index, columns=range(nsteps))
    return ql

def diffusive_routing_v02(
    connections,
    rconn,
    reaches_bytw,
    compute_func,
    parallel_compute_method,
    subnetwork_target_size,
    cpu_pool,
    nts,
    qts_subdivisions,
    independent_networks,
    param_df,
    qlats,
    q0,
    assume_short_ts,
):
    start_time = time.time()

    if parallel_compute_method == "by-network":
        with Parallel(n_jobs=cpu_pool, backend="threading") as parallel:
            jobs = []
            for twi, (tw, reach_list) in enumerate(reaches_bytw.items(), 1):
                segs = list(chain.from_iterable(reach_list))
                param_df_sub = param_df.loc[
                    segs, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
                ].sort_index()
                qlat_sub = qlats.loc[segs].sort_index()
                q0_sub = q0.loc[segs].sort_index()
                jobs.append(
                    delayed(compute_func)(
                        nts,
                        qts_subdivisions,
                        reach_list,
                        independent_networks[tw],
                        param_df_sub.index.values,
                        param_df_sub.columns.values,
                        param_df_sub.values,
                        qlat_sub.values,
                        q0_sub.values,
                        {},
                        assume_short_ts,
                    )
                )
            results = parallel(jobs)

    else:  # Execute in serial
        results = []
        for twi, (tw, reach_list) in enumerate(reaches_bytw.items(), 1):
            segs = list(chain.from_iterable(reach_list))
            param_df_sub = param_df.loc[
                segs, ["dt", "bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0"]
            ].sort_index()
            qlat_sub = qlats.loc[segs].sort_index()
            q0_sub = q0.loc[segs].sort_index()
            
            if tw==8777215:
                print(f"tw:{tw} reach_list{reach_list}")
                results.append(
                    compute_func(
                        nts,
                        qts_subdivisions,
                        reach_list,
                        independent_networks[tw],
                        param_df_sub.index.values,
                        param_df_sub.columns.values,
                        param_df_sub.values,
                        qlat_sub.values,
                        q0_sub.values,
                        {},
                        assume_short_ts,
                    )
                )

    return results
"""
def diffusive_input_data_v02(connections
                            , rconn
                            , reaches_bytw
                            , diffusive_parameters
                            , param_df
                            , qlats
                            ):

    start_time= time.time()
    
    import itertools
    import RouteLink_adjustment_v02 as rladj
    import fortran_python_map_v02 as fpm
    #from pyuniflowtzlt import uniflow_lookuptable

    usgs_retrievaltool_path= diffusive_parameters.get("usgs_retrievaltool_path",None)
    sys.path.append(usgs_retrievaltool_path)
    #results = []
    # diffusive time steps info.
    dt_ql_g=diffusive_parameters.get("dt_qlat",None) # time step of lateral flow
    dt_ub_g=diffusive_parameters.get("dt_upstream_boundary",None) # time step of us.boundary data
    dt_db_g=diffusive_parameters.get("dt_downstream_boundary",None) # time step of ds.boundary data
    saveinterval_g=diffusive_parameters.get("dt_output",None) # time step for outputting routed results
    saveinterval_ev_g=diffusive_parameters.get("dt_output",None) # time step for evaluating routed results
    dtini_g=diffusive_parameters.get("dt_diffusive",None) # initial simulation time step
    t0_g=0.0 #simulation start hr **set to zero for Fortran computation 
    tfin_g=diffusive_parameters.get("simulation_end_hr",None) # simulation end time

    # USGS data related info.
    usgsID= diffusive_parameters.get("usgsID",None)
    seg2usgsID= diffusive_parameters.get("link2usgsID",None)
    usgssDT= diffusive_parameters.get("usgs_start_date",None)
    usgseDT= diffusive_parameters.get("usgs_end_date",None)
    usgspCd= diffusive_parameters.get("usgs_parameterCd",None)

    # diffusive parameters
    cfl_g= diffusive_parameters.get("courant_number_upper_limit",None)
    theta_g= diffusive_parameters.get("theta_parameter",None)
    tzeq_flag_g= diffusive_parameters.get("chgeo_computation_flag",None)
    y_opt_g= diffusive_parameters.get("water_elevation_computation_flag",None)
    so_llm_g= diffusive_parameters.get("bed_slope_lower_limit",None)

    for twi, (tw, reach_list) in enumerate(reaches_bytw.items(), 1):
    
        if tw==933020089:  #if tw==8777215 or tw==166737669:
            # downstream boundary (tw) segment ID -> make an additional fake tw segment
            dbfksegID= str(tw)+ str(2)
            dbfksegID= int(dbfksegID) 
            ordered_reaches={}
            rchhead_reaches={}
            rchbottom_reaches={}
            z_all={}
    
            flat_list=list(itertools.chain(*reaches_bytw[tw])) # a list of all segments of reaches_bytw for a given tw.
            rconn_tw={key:value for key, value in rconn.items() if key in flat_list} # subset rconn by flat_list
            connections_tw= {key:value for key, value in connections.items() if key in flat_list} # subset connections by flat_list

            path_func = partial(nhd_network.split_at_junction, rconn_tw)
            tr = nhd_network.dfs_decomposition_depth_tuple(rconn_tw, path_func)
            jorder_reaches_tw=sorted(tr, key=lambda x: x[0]) # [ (jorder:[segments]), ... , (jorder:[segments]) ] 

            mx_jorder_tw=max(jorder_reaches_tw)[0] # maximum junction order of subnetwork of TW
            nrch_g=len(jorder_reaches_tw) # the number of reaches        
            maxlist=max(jorder_reaches_tw, key=lambda i:len(i[1]))
            mxncomp_g= len(maxlist[1])+1 # max. number of nodes (segments+one additional segment) within a reach

            for i in jorder_reaches_tw:
                # add one more segment(fake) to the end of a list of segments to account for node configuration.
                fksegID= i[1][len(i[1])-1]
                fksegID= int(str(fksegID) + str(2))
                i[1].append(fksegID)
                # additional segment(fake) to upstream bottom segments
                fk_usbseg=[int(str(x)+str(2)) for x in rconn_tw[i[1][0]]]            

                if i[0] not in ordered_reaches:
                    ordered_reaches.update({i[0]:[]})
                ordered_reaches[i[0]].append([i[1][0],{'number_segments':len(i[1]),\
                                            'segments_list':i[1],\
                                            'upstream_bottom_segments':fk_usbseg,\
                                            'downstream_head_segment':connections_tw[i[1][len(i[1])-2]]}]) 

                if i[1][0] not in rchhead_reaches:    
                    # a list of segments for a given head segment
                    rchhead_reaches.update({i[1][0]:{"number_segments":len(i[1]),\
                                                "segments_list":i[1]}})
                    # a list of segments for a given bottom segment
                    rchbottom_reaches.update({i[1][len(i[1])-1]:{"number_segments":len(i[1]),\
                                                         "segments_list":i[1]}})
                # for channel altitude adjustment
                z_all.update({seg:{'adj.alt':np.zeros(1)}
                                            for seg in i[1]})
            # cahnnel geometry data
            ch_geo_data_tw = param_df.loc[
            flat_list, ["bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0", "alt"]]
            ch_geo_data_tw[:]["cs"]= 1.0/ch_geo_data_tw[:]["cs"]
        #--------------------------------------------------------------------------------------
        #                                 Step 0-3           

        #    Adjust altitude so that altitude of the last sement of a reach is equal to that 
        #    of the first segment of its downstream reach right after their common junction.
        #--------------------------------------------------------------------------------------
            rladj.adj_alt1(mx_jorder_tw
                        , ordered_reaches
                        , ch_geo_data_tw
                        , dbfksegID
                        , z_all
                        ) 
        #--------------------------------------------------------------------------------------
        #                                 Step 0-4           

        #     Make Fortran-Python channel network mapping variables.
        #--------------------------------------------------------------------------------------   
            pynw={}
            frj=-1
            for x in range(mx_jorder_tw,-1,-1): 
                for head_segment, reach in ordered_reaches[x]:
                    frj= frj+1
                    pynw[frj]=head_segment

            #frnw_col=8
            frnw_col= diffusive_parameters.get("fortran_nework_map_col_number",None)
            frnw_g=fpm.fp_network_map(mx_jorder_tw
                    , ordered_reaches
                    , rchbottom_reaches
                    , nrch_g
                    , frnw_col
                    , dbfksegID
                    , pynw
                    )  
            #covert data type from integer to float for frnw
            dfrnw_g=np.zeros((nrch_g,frnw_col), dtype=float)
            for j in range(0,nrch_g):
                for col in range(0,frnw_col):
                    dfrnw_g[j,col]=float(frnw_g[j,col])
        #---------------------------------------------------------------------------------
        #                              Step 0-5

        #                  Prepare channel geometry data           
        #---------------------------------------------------------------------------------    
            z_ar_g, bo_ar_g, traps_ar_g, tw_ar_g, twcc_ar_g, mann_ar_g, manncc_ar_g, so_ar_g, dx_ar_g= fpm.fp_chgeo_map(mx_jorder_tw
                        , ordered_reaches
                        , ch_geo_data_tw
                        , z_all
                        , mxncomp_g
                        , nrch_g                    
                        )   
        #---------------------------------------------------------------------------------
        #                              Step 0-6

        #                  Prepare lateral inflow data           
        #---------------------------------------------------------------------------------
            segs = list(chain.from_iterable(reach_list))
            qlat_tw = qlats.loc[segs]    
            #tfin_g=len(qlat_tw.columns)-1 #entire simulation period in hrs
            nts_ql_g= int((tfin_g-t0_g)*3600.0/dt_ql_g)+1 # the number of the entire time steps of lateral flow data 

            qlat_g=np.zeros((nts_ql_g, mxncomp_g, nrch_g)) 

            fpm.fp_qlat_map(mx_jorder_tw
                , ordered_reaches
                , nts_ql_g
                , qlat_tw            
                , qlat_g
                ) 
        #---------------------------------------------------------------------------------
        #                              Step 0-7

        #       Prepare upstream boundary (top segments of head basin reaches) data            
        #---------------------------------------------------------------------------------
            nts_ub_g= nts_ql_g 
            ubcd_g = fpm.fp_ubcd_map(frnw_g
                                    , pynw
                                    , nts_ub_g
                                    , nrch_g
                                    , ch_geo_data_tw
                                    , qlat_tw
                                    , qlat_g
                                    )
        #---------------------------------------------------------------------------------
        #                              Step 0-8

        #       Prepare downstrea boundary (bottom segments of TW reaches) data            
        #---------------------------------------------------------------------------------        
            #import pdb; pdb.set_trace()
            if tw in seg2usgsID:
                ipos= seg2usgsID.index(tw)
                usgsID2tw= usgsID[ipos]         
                nts_db_g, dbcd_g=fpm.fp_dbcd_map(usgsID2tw
                            , usgssDT
                            , usgseDT
                            , usgspCd
                            )
            else:
                # no usgs data available at this TW.
                nts_db_g=-1.0
                
        #---------------------------------------------------------------------------------
        #                              Step 0-8

        #                 Prepare uniform flow lookup tables            
        #---------------------------------------------------------------------------------          
            #nhincr_m_g=20
            #nhincr_f_g=20               
            #timesdepth_g=10
            nhincr_m_g= diffusive_parameters.get("normaldepth_lookuptable_main_increment_number",None) 
            nhincr_f_g= diffusive_parameters.get("normaldepth_lookuptable_floodplain_increment_number",None) 
            timesdepth_g= diffusive_parameters.get("normaldepth_lookuptable_depth_multiplier",None)
            ufqlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))                        
            ufhlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))
            ufqlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))
            ufhlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))
            #ufhlt_m_g, ufqlt_m_g, ufhlt_f_g, ufqlt_f_g= uniflow_lookuptable(mxncomp_g 
            #                                                            , nrch_g 
            #                                                            , bo_ar_g 
            #                                                            , traps_ar_g 
            #                                                            , tw_ar_g 
            #                                                            , twcc_ar_g 
            #                                                            , mann_ar_g 
            #                                                            , manncc_ar_g 
            #                                                            , so_ar_g 
            #                                                            , nhincr_m_g 
            #                                                            , nhincr_f_g               
            #                                                            , frnw_col 
            #                                                            , dfrnw_g 
            #                                                            , timesdepth_g)

        #---------------------------------------------------------------------------------
        #                              Step 0-9

        #                       Run diffusive model            
        #---------------------------------------------------------------------------------  
            ntss_ev_g= int((tfin_g - t0_g)*3600.0/saveinterval_ev_g)+1 
            q_ev_g=np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
            elv_ev_g=np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
            
            # build a dictionary of diffusive model inputs
            diff_ins = {}
            diff_ins["dtini_g"] = dtini_g
            diff_ins["t0_g"] = t0_g
            diff_ins["tfin_g"] = tfin_g
            diff_ins["saveinterval_g"] = saveinterval_g
            diff_ins["saveinterval_ev_g"] = saveinterval_ev_g
            diff_ins["dt_ql_g"] = dt_ql_g
            diff_ins["dt_ub_g"] = dt_ub_g
            diff_ins["dt_db_g"] = dt_db_g
            diff_ins["nts_ql_g"] = nts_ql_g
            diff_ins["nts_ub_g"] = nts_ub_g
            diff_ins["nts_db_g"] = nts_db_g
            diff_ins["mxncomp_g"] = mxncomp_g
            diff_ins["nrch_g"] = nrch_g
            diff_ins["z_ar_g"] = z_ar_g
            diff_ins["bo_ar_g"] = bo_ar_g
            diff_ins["traps_ar_g"] = traps_ar_g
            diff_ins["tw_ar_g"] = tw_ar_g
            diff_ins["twcc_ar_g"] = twcc_ar_g
            diff_ins["mann_ar_g"] = mann_ar_g
            diff_ins["manncc_ar_g"] = manncc_ar_g
            diff_ins["so_ar_g"] = so_ar_g
            diff_ins["dx_ar_g"] = dx_ar_g
            diff_ins["nhincr_m_g"] = nhincr_m_g
            diff_ins["nhincr_f_g"] = nhincr_f_g
            diff_ins["ufhlt_m_g"] = ufhlt_m_g
            diff_ins["ufqlt_m_g"] = ufqlt_m_g
            diff_ins["ufhlt_f_g"] = ufhlt_f_g
            diff_ins["ufqlt_f_g"] = ufqlt_f_g
            diff_ins["frnw_col"] = frnw_col
            diff_ins["frnw_g"] = frnw_g
            diff_ins["qlat_g"] = qlat_g
            diff_ins["ubcd_g"] = ubcd_g
            diff_ins["dbcd_g"] = dbcd_g
            diff_ins["cfl_g"] = cfl_g
            diff_ins["theta_g"] = theta_g
            diff_ins["tzeq_flag_g"] = tzeq_flag_g
            diff_ins["y_opt_g"] = y_opt_g
            diff_ins["so_llm_g"] = so_llm_g
            diff_ins["ntss_ev_g"] = ntss_ev_g
            
            # save input data as yaml
            import yaml
            with open('diff_inputs.yml', 'w') as outfile:
                yaml.dump(diff_ins, outfile, default_flow_style=False)
            
    print("input data preparation for diffusive routing complete")
    print("... in %s seconds." % (time.time() - start_time))

def _input_handler():

    #args = _handle_args()

    #custom_input_file = args.custom_input_file
    custom_input_file="../../test/input/yaml/CustomInput_florence_933020089_dt300.yaml"
    
    supernetwork_parameters = {}
    waterbody_parameters = {}
    forcing_parameters = {}
    restart_parameters = {}
    output_parameters = {}
    run_parameters = {}
    parity_parameters = {}
    diffusive_parameters={}

    if custom_input_file:
        (
            supernetwork_parameters,
            waterbody_parameters,
            forcing_parameters,
            restart_parameters,
            output_parameters,
            run_parameters,
            parity_parameters,
            diffusive_parameters,
        ) = nhd_io.read_custom_input(custom_input_file)
        run_parameters["debuglevel"] *= -1
        print(f"in here: custom_input_file")

    else:
        print(f"deleted here")
    
    return (
        supernetwork_parameters,
        waterbody_parameters,
        forcing_parameters,
        restart_parameters,
        output_parameters,
        run_parameters,
        parity_parameters,
        diffusive_parameters,
    )


def main():
     
    (
        supernetwork_parameters,
        waterbody_parameters,
        forcing_parameters,
        restart_parameters,         
        output_parameters,
        run_parameters,
        parity_parameters,
        diffusive_parameters,
    ) = _input_handler()
   
    dt = run_parameters.get("dt", None)
    nts = run_parameters.get("nts", None)
    verbose = run_parameters.get("verbose", None)
    showtiming = run_parameters.get("showtiming", None)
    debuglevel = run_parameters.get("debuglevel", 0)
    

    geo_file_path = supernetwork_parameters.get("geo_file_path", None)
    print(f"geo_file_path:{geo_file_path}")    

    if verbose:
        print("creating supernetwork connections set")
    if showtiming:
        start_time = time.time()

    # STEP 1: Build basic network connections graph
    print(supernetwork_parameters)
    connections, param_df = nnu.build_connections(supernetwork_parameters, dt)
    #print(connections)
     
    if verbose:
        print("supernetwork connections set complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

    # STEP 2: Identify Independent Networks and Reaches by Network
    if showtiming:
        start_time = time.time()
    if verbose:
        print("organizing connections into reaches ...")

    independent_networks, reaches_bytw, rconn = nnu.organize_independent_networks(
        connections
    )

    if verbose:
        print("reach organization complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

    # STEP 4: Handle Channel Initial States
    if showtiming:
        start_time = time.time()
    if verbose:
        print("setting channel initial states ...")

    q0 = nnu.build_channel_initial_state(restart_parameters, param_df.index)

    if verbose:
        print("channel initial states complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))
        start_time = time.time()

    # STEP 5: Read (or set) QLateral Inputs
    if showtiming:
        start_time = time.time()
    if verbose:
        print("creating qlateral array ...")

    qlats = nnu.build_qlateral_array(forcing_parameters, connections.keys(), nts)

    if verbose:
        print("qlateral array complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

    ################### Main Execution Loop across ordered networks
    if showtiming:
        main_start_time = time.time()
    if verbose:
        print(f"executing routing computation ...")
        
    
    # Prepare input datasets for diffusive routing
    diffusive_input_data_v02(connections
                            , rconn
                            , reaches_bytw
                            , diffusive_parameters
                            , param_df 
                            , qlats
                            )

    #if run_parameters.get("compute_method", None) == "standard cython compute network":
    #    compute_func = mc_reach.compute_network
    #else:
    #    compute_func = mc_reach.compute_network

    #results = diffusive_routing_v02(
    #    connections,
    #    rconn,
    #    reaches_bytw,
    #    compute_func,
    #    run_parameters.get("parallel_compute_method", None),
    #    run_parameters.get("subnetwork_target_size", 1),
        # The default here might be the whole network or some percentage...
    #    run_parameters.get("cpu_pool", None),
    #    run_parameters.get("nts", 1),
    #    run_parameters.get("qts_subdivisions", 1),
    #    independent_networks,
    #    param_df,
    #    qlats,
    #    q0,
    #    run_parameters.get("assume_short_ts", False),
    #)

    #csv_output_folder = output_parameters.get("csv_output_folder", None)
    #if (debuglevel <= -1) or csv_output_folder:
    #    qvd_columns = pd.MultiIndex.from_product(
    #        [range(nts), ["q", "v", "d"]]
    #    ).to_flat_index()
    #    flowveldepth = pd.concat(
    #        [pd.DataFrame(d, index=i, columns=qvd_columns) for i, d in results],
    #        copy=False,
    #    )

    #    if csv_output_folder:
    #        flowveldepth = flowveldepth.sort_index()
    #        output_path = pathlib.Path(csv_output_folder).resolve()
    #        flowveldepth.to_csv(output_path.joinpath(f"{args.supernetwork}.csv"))

    #    if debuglevel <= -1:
    #        print(flowveldepth)

    if verbose:
        print("ordered reach computation complete")
    if showtiming:
        print("... in %s seconds." % (time.time() - start_time))

if __name__ == "__main__":
    main()


In [ ]:
qlats